In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from model_mo2cap2 import mo2cap_ours
from tqdm import tqdm
import os

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = TempModel(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		image = test_batch['image'].cuda()
		image_zoom = test_batch['image_zoom'].cuda()
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {
			'image':image,
			# 'image_zoom' : image_zoom,
			}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp812'+'/*'))
path_ = ['/workspace/2d_to_3d/apps/exp812/8.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	_path = f'/workspace/2d_to_3d/apps/eval/ours_depth_sil_fusion_2_test'
	_name = f'baseline812_{i}.json'
	if not os.path.exists(_path):
		os.makedirs(_path)

	main(model_path=p,output_path=os.path.join(_path,_name))

03:24:00 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 75.04975891113281, 'UpperStretching': 78.06291961669922, 'Walking': 75.1675033569336, 'LowerStretching': 73.4614486694336, 'Reacting': 57.40007019042969, 'Patting': 78.93453979492188, 'Gesticuling': 55.683074951171875, 'Gaming': 65.68125915527344, 'Talking': 53.611671447753906, 'Greeting': 52.251407623291016}, 'UpperBody': {'All': 57.03192901611328, 'UpperStretching': 62.767127990722656, 'Walking': 49.387672424316406, 'LowerStretching': 47.09769058227539, 'Reacting': 47.76639175415039, 'Patting': 58.37625503540039, 'Gesticuling': 50.44765853881836, 'Gaming': 49.112281799316406, 'Talking': 46.997894287109375, 'Greeting': 45.71799087524414}, 'LowerBody': {'All': 93.06758117675781, 'UpperStretching': 93.35871124267578, 'Walking': 100.94734954833984, 'LowerStretching': 99.82520294189453, 'Reacting': 67.03375244140625, 'Patting': 99.49283599853516, 'Gesticuling': 60.918495178222656, 'Gaming': 82.250244140625, 'Talking': 60.22545623779297, 'Greeting': 58.78483200073242}}

  7%|▋         | 401/5352 [01:03<10:25,  7.92it/s]

{'FullBody': {'All': 75.33952331542969, 'UpperStretching': 78.7299575805664, 'Walking': 74.83193969726562, 'LowerStretching': 74.15045166015625, 'Reacting': 59.5658073425293, 'Patting': 72.12612915039062, 'Gesticuling': 56.16087341308594, 'Gaming': 63.413692474365234, 'Talking': 50.53632736206055, 'Greeting': 49.045562744140625}, 'UpperBody': {'All': 57.47309494018555, 'UpperStretching': 63.649723052978516, 'Walking': 49.20978546142578, 'LowerStretching': 47.37191390991211, 'Reacting': 51.371952056884766, 'Patting': 53.65135955810547, 'Gesticuling': 48.93104934692383, 'Gaming': 47.01490020751953, 'Talking': 44.84604263305664, 'Greeting': 42.546138763427734}, 'LowerBody': {'All': 93.2059555053711, 'UpperStretching': 93.81018829345703, 'Walking': 100.45409393310547, 'LowerStretching': 100.92899322509766, 'Reacting': 67.75967407226562, 'Patting': 90.60088348388672, 'Gesticuling': 63.39068603515625, 'Gaming': 79.81246948242188, 'Talking': 56.22661209106445, 'Greeting': 55.544986724853516}}

  8%|▊         | 429/5352 [01:06<10:20,  7.93it/s]

  8%|▊         | 435/5352 [01:07<10:29,  7.81it/s]